# HEC-RAS 2D HDF Data Analysis Notebook

This notebook demonstrates how to manipulate and analyze HEC-RAS 2D HDF data using the ras-commander library. It leverages the HdfBase, HdfUtils, HdfStruc, HdfMesh, HdfXsec, HdfBndry, HdfPlan, HdfResultsPlan, HdfResultsMesh, and HdfResultsXsec classes to streamline data extraction, processing, and visualization.


In [1]:
# Import required Libraries
import subprocess
import sys
import os
from pathlib import Path

def install_module(module_name):
    try:
        __import__(module_name)
    except ImportError:
        print(f"{module_name} not found. Installing...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-U", module_name])

# List of modules to check and install if necessary
modules = ['h5py', 'numpy', 'requests', 'geopandas', 'matplotlib', 'pandas', 'pyproj', 'shapely', 'xarray', 'dask']
for module in modules:
    install_module(module)

# Import the rest of the required libraries
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import pyproj
from shapely.geometry import Point, LineString, Polygon
import xarray as xr
import dask.array as da

## Importing ras-commander flexibly (from package or local dev copy)

In [2]:
import sys
from pathlib import Path

# Flexible imports to allow for development without installation 
#  ** Use this version with Jupyter Notebooks **
try:
    # Try to import from the installed package
    from ras_commander import (
        init_ras_project, 
        HdfBase, 
        HdfUtils, 
        HdfStruc, 
        HdfMesh, 
        HdfXsec, 
        HdfBndry, 
        HdfPlan, 
        HdfResultsPlan, 
        HdfResultsMesh, 
        HdfResultsXsec,
        RasExamples, 
        RasCmdr, 
        RasPlan, 
        RasGeo, 
        RasUnsteady, 
        RasUtils, 
        RasPrj, 
        RasGpt, 
        ras,
        XsSteadyOutputVar,
        SummaryOutputVar,
        TimeSeriesOutputVar
    )
    from ras_commander.Decorators import standardize_input, log_call
    from ras_commander.LoggingConfig import setup_logging, get_logger
except ImportError:
    # If the import fails, add the parent directory to the Python path
    import os
    current_file = Path(os.getcwd()).resolve()
    parent_directory = current_file.parent
    sys.path.append(str(parent_directory))
    
    # Now try to import again
    from ras_commander import (
        init_ras_project, 
        HdfBase, 
        HdfUtils, 
        HdfStruc, 
        HdfMesh, 
        HdfXsec, 
        HdfBndry, 
        HdfPlan, 
        HdfResultsPlan, 
        HdfResultsMesh, 
        HdfResultsXsec,
        RasExamples, 
        RasCmdr, 
        RasPlan, 
        RasGeo, 
        RasUnsteady, 
        RasUtils, 
        RasPrj, 
        RasGpt, 
        ras,
    )
    from ras_commander.Decorators import standardize_input, log_call
    from ras_commander.LoggingConfig import setup_logging, get_logger

print("ras_commander imported successfully")

ras_commander imported successfully


In [3]:
# Download the BaldEagleCrkMulti2D project from HEC and run plan 01

# Define the path to the BaldEagleCrkMulti2D project
current_dir = Path.cwd()  # Adjust if your notebook is in a different directory
bald_eagle_path = current_dir / "example_projects" / "BaldEagleCrkMulti2D"
import logging

# Check if BaldEagleCrkMulti2D.p06.hdf exists (so we don't have to re-run the simulation when re-running or debugging)
hdf_file = bald_eagle_path / "BaldEagleDamBrk.p06.hdf"

if not hdf_file.exists():
    # Initialize RasExamples and extract the BaldEagleCrkMulti2D project
    ras_examples = RasExamples()
    ras_examples.extract_project(["BaldEagleCrkMulti2D"])

    # Initialize custom Ras object
    bald_eagle = RasPrj()

    # Initialize the RAS project using the custom ras object
    bald_eagle = init_ras_project(bald_eagle_path, "6.6", ras_instance=bald_eagle)
    logging.info(f"Bald Eagle project initialized with folder: {bald_eagle.project_folder}")
    
    logging.info(f"Bald Eagle object id: {id(bald_eagle)}")
    
    # Define the plan number to execute
    plan_number = "06"

    # Set plan keys for the project
    RasPlan.update_plan_value(plan_number, "Run HTab", 1, ras_object=bald_eagle)
    RasPlan.update_plan_value(plan_number, "Run UNet", 1, ras_object=bald_eagle)
    RasPlan.update_plan_value(plan_number, "Run PostProcess", 1, ras_object=bald_eagle)
    RasPlan.update_plan_value(plan_number, "Run RASMapper", 0, ras_object=bald_eagle)

    # Execute Plan 06 using RasCmdr for Bald Eagle
    print(f"Executing Plan {plan_number} for the Bald Eagle Creek project...")
    success_bald_eagle = RasCmdr.compute_plan(plan_number, ras_object=bald_eagle)
    if success_bald_eagle:
        print(f"Plan {plan_number} executed successfully for Bald Eagle.\n")
    else:
        print(f"Plan {plan_number} execution failed for Bald Eagle.\n")
else:
    print("BaldEagleCrkMulti2D.p06.hdf already exists. Skipping project extraction and plan execution.")
    # Initialize the RAS project using the custom ras object
    bald_eagle = RasPrj()
    bald_eagle = init_ras_project(bald_eagle_path, "6.6", ras_instance=bald_eagle)
    plan_number = "06"

2024-10-15 06:29:50,592 - ras_commander.RasExamples - INFO - Example projects folder: c:\GH\ras-commander\examples\example_projects
2024-10-15 06:29:50,595 - ras_commander.RasExamples - WARNING - No existing example projects zip file found.
2024-10-15 06:29:50,596 - ras_commander.RasExamples - INFO - No example projects zip file found. Downloading...
2024-10-15 06:29:50,597 - ras_commander.RasExamples - INFO - Getting example projects for version 6.6
2024-10-15 06:29:50,599 - ras_commander.RasExamples - INFO - Downloading HEC-RAS Example Projects from https://github.com/HydrologicEngineeringCenter/hec-downloads/releases/download/1.0.33/Example_Projects_6_6.zip. 
The file is over 400 MB, so it may take a few minutes to download....
2024-10-15 06:30:08,071 - ras_commander.RasExamples - INFO - Downloaded to c:\GH\ras-commander\examples\Example_Projects_6_6.zip
2024-10-15 06:30:08,072 - ras_commander.RasExamples - INFO - Found zip file: c:\GH\ras-commander\examples\Example_Projects_6_6.zip

In [ ]:
# Load Plan and Geometry Dataframes and find Plan and Geometry HDF Paths

# Display plan_df for bald_eagle project
print("Plan DataFrame for bald_eagle project:")
display(bald_eagle.plan_df)

# Display geom_df for bald_eagle project
print("\nGeometry DataFrame for bald_eagle project:")
display(bald_eagle.geom_df)

# Get the plan HDF path
plan_number = "06"  # Assuming we're using plan 01 as in the previous code
plan_hdf_path = bald_eagle.plan_df.loc[bald_eagle.plan_df['plan_number'] == plan_number, 'HDF_Results_Path'].values[0]

# Get the geometry file number from the plan DataFrame
geom_file = bald_eagle.plan_df.loc[bald_eagle.plan_df['plan_number'] == plan_number, 'Geom File'].values[0]
geom_number = geom_file[1:]  # Remove the 'g' prefix

# Get the geometry HDF path
geom_hdf_path = bald_eagle.geom_df.loc[bald_eagle.geom_df['geom_number'] == geom_number, 'hdf_path'].values[0]

print(f"\nPlan HDF path for Plan {plan_number}: {plan_hdf_path}")
print(f"Geometry HDF path for Plan {plan_number}: {geom_hdf_path}")

In [5]:
# Define the HDF input path as Plan Number

plan_number = "06"  # Assuming we're using plan 01 as in the previous code


RasHdfUtils
| Method Name | Description |
|-------------|-------------|
| get_attrs | Converts attributes from a HEC-RAS HDF file into a Python dictionary for a given attribute path |
| get_root_attrs | Returns attributes at root level of HEC-RAS HDF file |
| get_hdf_paths_with_properties | Gets all paths in the HDF file with their properties |
| get_group_attributes_as_df | Gets attributes of a group in the HDF file as a DataFrame |
| get_hdf_filename | Gets the HDF filename from various input types |
| get_runtime_data | Extracts runtime and compute time data from a single HDF file |


In [ ]:
# Get HDF Paths with Properties (For Exploring HDF Files)
plan_number = "06"  # Assuming we're using plan 06 as in the previous code
hdf_paths_df = HdfUtils.get_hdf_paths_with_properties(plan_number, ras_object=bald_eagle)
display(hdf_paths_df.head())

In [ ]:
# Example: Extract runtime and compute time data
print("\nExample 2: Extracting runtime and compute time data")
runtime_df = HdfResultsPlan.get_runtime_data(hdf_input=plan_number, ras_object=bald_eagle)
if runtime_df is not None:
    display(runtime_df)
else:
    print("No runtime data found.")

In [8]:
# TODO: Example for get_attrs

In [9]:
# TODO: Example for get_root_attrs

In [10]:
# TODO: Example for get_hdf_paths_with_properties

In [11]:
# TODO: Example for get_group_attributes_as_df

Table of all the functions in the RasGeomHdf class from the ras_commander/RasGeomHdf.py file:

| Function Name | Description |
|---------------|-------------|
| projection | Returns the projection of the RAS geometry as a pyproj.CRS object |
| get_geom_attrs | Returns base geometry attributes from a HEC-RAS HDF file |

| mesh_area_names | Returns a list of the 2D mesh area names of the RAS geometry |
| get_geom_2d_flow_area_attrs | Returns geometry 2d flow area attributes from a HEC-RAS HDF file |
| mesh_areas | Returns 2D flow area perimeter polygons |
| mesh_cell_polygons | Returns 2D flow mesh cell polygons |
| mesh_cell_points | Returns 2D flow mesh cell points |
| mesh_cell_faces | Returns 2D flow mesh cell faces |

| get_geom_structures_attrs | Returns geometry structures attributes from a HEC-RAS HDF file |




| bc_lines | Returns 2D mesh area boundary condition lines |
| breaklines | Returns 2D mesh area breaklines |



| refinement_regions | Returns 2D mesh area refinement regions |
| structures | Returns the model structures |
| reference_lines_names | Returns reference line names |
| reference_points_names | Returns reference point names |
| reference_lines | Returns the reference lines geometry and attributes |
| reference_points | Returns the reference points geometry and attributes |
| cross_sections | Returns the model 1D cross sections |
| river_reaches | Returns the model 1D river reach lines |
| cross_sections_elevations | Returns the model cross section elevation information |

In [ ]:
# For all of the RasGeomHdf Class Functions, we will use geom_hdf_path
print(geom_hdf_path)

# For the example project, plan 06 is associated with geometry 09
# If you want to call the geometry by number, call RasHdfGeom functions with a number
# Otherwise, if you want to look up geometry hdf path by plan number, follow the logic in the previous code cells

In [ ]:
# Use HdfUtils for extracting projection
print("\nExtracting Projection from HDF")
projection = HdfUtils.projection(hdf_path=geom_hdf_path)
if projection:
    print(f"Projection: {projection}")
else:
    print("No projection information found.")

In [ ]:
# Use HdfPlan for geometry-related operations
print("\nExample: Extracting Base Geometry Attributes")
geom_attrs = HdfPlan.get_geom_attrs(geom_hdf_path, ras_object=bald_eagle)

if geom_attrs:
    # Convert the dictionary to a DataFrame for better display
    geom_attrs_df = pd.DataFrame([geom_attrs])
    
    # Display the DataFrame
    print("Base Geometry Attributes:")
    display(geom_attrs_df)
else:
    print("No base geometry attributes found.")


In [ ]:
# Use HdfMesh for geometry-related operations
print("\nExample 3: Listing 2D Flow Area Names")
flow_area_names = HdfMesh.mesh_area_names(geom_hdf_path, ras_object=bald_eagle)
print("2D Flow Area Names:", flow_area_names)

In [ ]:
# Example: Get 2D Flow Area Attributes (get_geom_2d_flow_area_attrs)
print("\nExample: Extracting 2D Flow Area Attributes")
flow_area_attributes = HdfMesh.get_geom_2d_flow_area_attrs(geom_hdf_path, ras_object=bald_eagle)

if flow_area_attributes:
    # Convert the dictionary to a DataFrame for better display
    flow_area_df = pd.DataFrame([flow_area_attributes])
    
    # Display the DataFrame
    print("2D Flow Area Attributes:")
    display(flow_area_df)
    
    # Optionally, you can access specific attributes
    print("\nSpecific Attribute Examples:")
    print(f"Cell Average Size: {flow_area_attributes.get('Cell Average Size', 'N/A')}")
    print(f"Manning's n: {flow_area_attributes.get('Manning''s n', 'N/A')}")
    print(f"Terrain Filename: {flow_area_attributes.get('Terrain Filename', 'N/A')}")
else:
    print("No 2D Flow Area attributes found.")

# Note: This example assumes that get_geom_2d_flow_area_attrs returns a dictionary.
# If it returns a different format, you may need to adjust the code accordingly.


In [ ]:
# Example: Get 2D Flow Area Perimeter Polygons (mesh_areas)
print("\nExample: Extracting 2D Flow Area Perimeter Polygons")
mesh_areas = HdfMesh.mesh_areas(geom_hdf_path, ras_object=bald_eagle)

In [ ]:
# Plot the 2D Flow Area Perimeter Polygons
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(12, 8))
mesh_areas.plot(ax=ax, edgecolor='black', facecolor='none')

# Add labels for each polygon
for idx, row in mesh_areas.iterrows():
    centroid = row.geometry.centroid
    # Check if 'Name' column exists, otherwise use a default label
    label = row.get('Name', f'Area {idx}')
    ax.annotate(label, (centroid.x, centroid.y), ha='center', va='center')

plt.title('2D Flow Area Perimeter Polygons')
plt.xlabel('Easting')
plt.ylabel('Northing')
plt.tight_layout()
plt.show()

In [ ]:
# Example: Extract 2D Flow Area Attributes 
print("\nExample 4: Extracting 2D Flow Area Attributes")
flow_area_attributes = HdfMesh.get_geom_2d_flow_area_attrs(geom_hdf_path, ras_object=bald_eagle)
if flow_area_attributes:
    # Convert the dictionary to a DataFrame for better display
    flow_area_attributes_df = pd.DataFrame([flow_area_attributes])
    display(flow_area_attributes_df)
else:
    print("No 2D Flow Area attributes found.")

In [ ]:
# Example: Extract mesh cell faces
print("\nExample: Extracting mesh cell faces")

# Get mesh cell faces
mesh_cell_faces = HdfMesh.mesh_cell_faces(geom_hdf_path, ras_object=bald_eagle)

# Display the first few rows of the mesh cell faces DataFrame
print("First few rows of mesh cell faces:")
display(mesh_cell_faces.head())

# Plot the mesh cell faces
fig, ax = plt.subplots(figsize=(12, 8))

# Plot all cell faces
for _, row in mesh_cell_faces.iterrows():
    ax.plot(*row['geometry'].xy, color='blue', linewidth=0.5, alpha=0.5)

# Set plot title and labels
plt.title('Mesh Cell Faces')
plt.xlabel('Easting')
plt.ylabel('Northing')

# Add a colorbar to show face IDs
scatter = ax.scatter(
    mesh_cell_faces.geometry.centroid.x,
    mesh_cell_faces.geometry.centroid.y,
    c=mesh_cell_faces['face_id'],
    cmap='viridis',
    s=1,
    alpha=0.5
)
plt.colorbar(scatter, label='Face ID')

plt.tight_layout()
plt.show()

# Calculate and display some statistics
print("\nMesh Cell Faces Statistics:")
print(f"Total number of cell faces: {len(mesh_cell_faces)}")
print(f"Number of unique meshes: {mesh_cell_faces['mesh_name'].nunique()}")


In [ ]:
# Function to find the nearest cell face to a given point
def find_nearest_cell_face(point, cell_faces_df):
    """
    Find the nearest cell face to a given point.

    Args:
        point (shapely.geometry.Point): The input point.
        cell_faces_df (GeoDataFrame): DataFrame containing cell face linestrings.

    Returns:
        int: The face_id of the nearest cell face.
        float: The distance to the nearest cell face.
    """
    # Calculate distances from the input point to all cell faces
    distances = cell_faces_df.geometry.distance(point)

    # Find the index of the minimum distance
    nearest_index = distances.idxmin()

    # Get the face_id and distance of the nearest cell face
    nearest_face_id = cell_faces_df.loc[nearest_index, 'face_id']
    nearest_distance = distances[nearest_index]

    return nearest_face_id, nearest_distance

# Example usage
print("\nExample: Finding the nearest cell face to a given point")

# Create a sample point (you can replace this with any point of interest)
from shapely.geometry import Point
from geopandas import GeoDataFrame

# Get the projection from the geometry file
projection = HdfUtils.projection(hdf_path=geom_hdf_path)
if projection:
    print(f"Using projection: {projection}")
else:
    print("No projection information found. Using default CRS.")
    projection = "EPSG:4326"  # Default to WGS84 if no projection is found

# Create the sample point with the correct CRS
sample_point = GeoDataFrame({'geometry': [Point(2042250, 351750)]}, crs=projection)

if not mesh_cell_faces.empty and not sample_point.empty:
    # Ensure the CRS of the sample point matches the mesh_cell_faces
    if sample_point.crs != mesh_cell_faces.crs:
        sample_point = sample_point.to_crs(mesh_cell_faces.crs)
    
    nearest_face_id, distance = find_nearest_cell_face(sample_point.geometry.iloc[0], mesh_cell_faces)
    print(f"Nearest cell face to point {sample_point.geometry.iloc[0].coords[0]}:")
    print(f"Face ID: {nearest_face_id}")
    print(f"Distance: {distance:.2f} units")

    # Visualize the result
    fig, ax = plt.subplots(figsize=(12, 8))
    
    # Plot all cell faces
    mesh_cell_faces.plot(ax=ax, color='blue', linewidth=0.5, alpha=0.5, label='Cell Faces')
    
    # Plot the sample point
    sample_point.plot(ax=ax, color='red', markersize=100, alpha=0.7, label='Sample Point')
    
    # Plot the nearest cell face
    nearest_face = mesh_cell_faces[mesh_cell_faces['face_id'] == nearest_face_id]
    nearest_face.plot(ax=ax, color='green', linewidth=2, alpha=0.7, label='Nearest Face')
    
    # Set labels and title
    ax.set_xlabel('X Coordinate')
    ax.set_ylabel('Y Coordinate')
    ax.set_title('Nearest Cell Face to Sample Point')
    
    # Add legend and grid
    ax.legend()
    ax.grid(True)
    
    # Adjust layout and display
    plt.tight_layout()
    plt.show()
else:
    print("Unable to perform nearest cell face search due to missing data.")



In [ ]:
# Example: Extract Cell Polygons
print("\nExample 6: Extracting Cell Polygons")
cell_polygons_df = HdfMesh.mesh_cell_polygons(geom_hdf_path, ras_object=bald_eagle)
if not cell_polygons_df.empty:
    display(cell_polygons_df.head())
else:
    print("No Cell Polygons found.")

# Plot cell polygons
if not cell_polygons_df.empty:
    fig, ax = plt.subplots(figsize=(12, 8))
    
    # Plot cell polygons
    cell_polygons_df.plot(ax=ax, edgecolor='blue', facecolor='none')
    
    # Set labels and title
    ax.set_xlabel('X Coordinate')
    ax.set_ylabel('Y Coordinate')
    ax.set_title('2D Flow Area Cell Polygons')
    
    # Add grid
    ax.grid(True)
    
    # Adjust layout and display
    plt.tight_layout()
    plt.show()
else:
    print("No cell polygon data available for plotting.")

In [ ]:
# Example 5: Extract Cell Info
print("\nExample 5: Extracting Cell Info")
cell_info_df = HdfMesh.mesh_cell_points(geom_hdf_path, ras_object=bald_eagle)
if not cell_info_df.empty:
    display(cell_info_df.head())
else:
    print("No Cell Info found.")

# Plot cell centers
import matplotlib.pyplot as plt

if not cell_info_df.empty:
    fig, ax = plt.subplots(figsize=(12, 8))
    
    # Plot cell centers
    cell_info_df.plot(ax=ax, color='red', markersize=5)
    
    # Set labels and title
    ax.set_xlabel('X Coordinate')
    ax.set_ylabel('Y Coordinate')
    ax.set_title('2D Flow Area Cell Centers')
    
    # Add grid
    ax.grid(True)
    
    # Adjust layout and display
    plt.tight_layout()
    plt.show()
else:
    print("No cell data available for plotting.")


In [ ]:
# Provide function that will accept a geopandas point object and will find the nearest cell center
# Function to find the nearest cell center to a given point
def find_nearest_cell(point, cell_centers_df):
    """
    Find the nearest cell center to a given point.

    Args:
        point (shapely.geometry.Point): The input point.
        cell_centers_df (GeoDataFrame): DataFrame containing cell center points.

    Returns:
        int: The cell_id of the nearest cell.
        float: The distance to the nearest cell center.
    """
    # Calculate distances from the input point to all cell centers
    distances = cell_centers_df.geometry.distance(point)

    # Find the index of the minimum distance
    nearest_index = distances.idxmin()

    # Get the cell_id and distance of the nearest cell
    nearest_cell_id = cell_centers_df.loc[nearest_index, 'cell_id']
    nearest_distance = distances[nearest_index]

    return nearest_cell_id, nearest_distance

# Example usage
print("\nExample: Finding the nearest cell to a given point")

# Create a sample point (you can replace this with any point of interest)
from shapely.geometry import Point
from geopandas import GeoDataFrame

# Get the projection from the geometry file
projection = HdfUtils.projection(hdf_path=geom_hdf_path)
if projection:
    print(f"Using projection: {projection}")
else:
    print("No projection information found. Using default CRS.")
    projection = "EPSG:4326"  # Default to WGS84 if no projection is found

# Create the sample point with the correct CRS
sample_point = GeoDataFrame({'geometry': [Point(2083500, 370800)]}, crs=projection)

if not cell_info_df.empty and not sample_point.empty:
    # Ensure the CRS of the sample point matches the cell_info_df
    if sample_point.crs != cell_info_df.crs:
        sample_point = sample_point.to_crs(cell_info_df.crs)
    
    nearest_cell_id, distance = find_nearest_cell(sample_point.geometry.iloc[0], cell_info_df)
    print(f"Nearest cell to point {sample_point.geometry.iloc[0].coords[0]}:")
    print(f"Cell ID: {nearest_cell_id}")
    print(f"Distance: {distance:.2f} units")

    # Visualize the result
    fig, ax = plt.subplots(figsize=(12, 8))
    
    # Plot all cell centers
    cell_info_df.plot(ax=ax, color='blue', markersize=5, alpha=0.5, label='Cell Centers')
    
    # Plot the sample point
    sample_point.plot(ax=ax, color='red', markersize=100, alpha=0.7, label='Sample Point')
    
    # Plot the nearest cell center
    nearest_cell = cell_info_df[cell_info_df['cell_id'] == nearest_cell_id]
    nearest_cell.plot(ax=ax, color='green', markersize=100, alpha=0.7, label='Nearest Cell')
    
    # Set labels and title
    ax.set_xlabel('X Coordinate')
    ax.set_ylabel('Y Coordinate')
    ax.set_title('Nearest Cell to Sample Point')
    
    # Add legend and grid
    ax.legend()
    ax.grid(True)
    
    # Adjust layout and display
    plt.tight_layout()
    plt.show()
else:
    print("Unable to perform nearest cell search due to missing data.")


In [ ]:
# Get geometry structures attributes
print("\nGetting geometry structures attributes")
geom_structures_attrs = HdfStruc.get_geom_structures_attrs(geom_hdf_path, ras_object=bald_eagle)
if geom_structures_attrs:
    print("Geometry structures attributes:")
    for key, value in geom_structures_attrs.items():
        print(f"{key}: {value}")
else:
    print("No geometry structures attributes found.")

In [26]:
# TODO: Paths and Functions for each type of structure: 

# Getting geometry structures attributes
# Geometry structures attributes:
# Bridge/Culvert Count: 0
# Connection Count: 4
# Has Bridge Opening (2D): 0
# Inline Structure Count: 0
# Lateral Structure Count: 0

In [ ]:
# Example: Extract Boundary Condition Lines and Plot with 2D Flow Area Perimeter Polygons
print("\nExample 7: Extracting Boundary Condition Lines and Plotting with 2D Flow Area Perimeter Polygons")
bc_lines_df = HdfBndry.bc_lines(geom_hdf_path, ras_object=bald_eagle)
if not bc_lines_df.empty:
    display(bc_lines_df.head())
else:
    print("No Boundary Condition Lines found.")

# Plot if data exists
if not bc_lines_df.empty or not mesh_areas.empty:
    fig, ax = plt.subplots(figsize=(12, 8))
    
    # Plot 2D Flow Area Perimeter Polygons
    if not mesh_areas.empty:
        mesh_areas.plot(ax=ax, edgecolor='black', facecolor='none', alpha=0.7, label='2D Flow Area')
        
        # Add labels for each polygon
        for idx, row in mesh_areas.iterrows():
            centroid = row.geometry.centroid
            label = row.get('Name', f'Area {idx}')
            ax.annotate(label, (centroid.x, centroid.y), ha='center', va='center')
    
    # Plot boundary condition lines
    if not bc_lines_df.empty:
        bc_lines_df.plot(ax=ax, color='red', linewidth=2, label='Boundary Condition Lines')
    
    # Set labels and title
    ax.set_xlabel('Easting')
    ax.set_ylabel('Northing')
    ax.set_title('2D Flow Area Perimeter Polygons and Boundary Condition Lines')
    
    # Add grid and legend
    ax.grid(True)
    ax.legend()
    
    # Adjust layout and display
    plt.tight_layout()
    plt.show()
else:
    print("No data available for plotting.")

In [ ]:
# Example: Extract Breaklines and Plot with 2D Flow Area Perimeter Polygons
print("\nExample 8: Extracting Breaklines and Plotting with 2D Flow Area Perimeter Polygons")
breaklines_df = HdfBndry.breaklines(geom_hdf_path, ras_object=bald_eagle)
if not breaklines_df.empty:
    display(breaklines_df.head())
else:
    print("No Breaklines found.")

# Plot breaklines and 2D Flow Area Perimeter Polygons if they exist
if not breaklines_df.empty or not mesh_areas.empty:
    fig, ax = plt.subplots(figsize=(12, 8))
    
    # Plot 2D Flow Area Perimeter Polygons
    if not mesh_areas.empty:
        mesh_areas.plot(ax=ax, edgecolor='black', facecolor='none', alpha=0.7, label='2D Flow Area')
        
        # Add labels for each polygon
        for idx, row in mesh_areas.iterrows():
            centroid = row.geometry.centroid
            label = row.get('Name', f'Area {idx}')
            ax.annotate(label, (centroid.x, centroid.y), ha='center', va='center')
    
    # Plot breaklines
    if not breaklines_df.empty:
        breaklines_df.plot(ax=ax, color='blue', linewidth=2, label='Breaklines')
    
    # Set labels and title
    ax.set_xlabel('Easting')
    ax.set_ylabel('Northing')
    ax.set_title('2D Flow Area Perimeter Polygons and Breaklines')
    
    # Add grid and legend
    ax.grid(True)
    ax.legend()
    
    # Adjust layout and display
    plt.tight_layout()
    plt.show()
else:
    print("No data available for plotting.")

In [29]:
# INSTEAD OF hdf_input, USE plan_hdf_path or geom_hdf_path as appropriate 

In [ ]:
# Example: Get structures
structures_gdf = HdfStruc.structures(geom_hdf_path, ras_object=bald_eagle)
print("Structures:")
if not structures_gdf.empty:
    display(structures_gdf.head())
else:
    print("No structures found in the geometry file.")

In [ ]:
# Example: Get reference line names
ref_line_names = HdfBndry.reference_lines_names(geom_hdf_path, ras_object=bald_eagle)
print("\nReference Line Names:")
print(ref_line_names)

In [ ]:
# Example: Get reference point names
ref_point_names = HdfBndry.reference_points_names(geom_hdf_path, ras_object=bald_eagle)
print("\nReference Point Names:")
print(ref_point_names)

In [ ]:
# Example: Get reference lines
ref_lines_gdf = HdfBndry.reference_lines(geom_hdf_path, ras_object=bald_eagle)
print("\nReference Lines:")
if not ref_lines_gdf.empty:
    display(ref_lines_gdf.head())
else:
    print("No reference lines found in the geometry file.")

In [ ]:
# Example: Get reference points
ref_points_gdf = HdfBndry.reference_points(geom_hdf_path, ras_object=bald_eagle)
print("\nReference Points:")
if not ref_points_gdf.empty:
    display(ref_points_gdf.head())
else:
    print("No reference points found in the geometry file.")

In [ ]:
# Example: Get cross sections
cross_sections_gdf = HdfXsec.cross_sections(geom_hdf_path, ras_object=bald_eagle)
print("\nCross Sections:")
if not cross_sections_gdf.empty:
    display(cross_sections_gdf.head())
else:
    print("No cross sections found in the geometry file.")

In [ ]:
# Example: Get river reaches
river_reaches_gdf = HdfXsec.river_reaches(geom_hdf_path, ras_object=bald_eagle)
print("\nRiver Reaches:")
if not river_reaches_gdf.empty:
    display(river_reaches_gdf.head())
else:
    print("No river reaches found in the geometry file.")


In [ ]:
# Example: Get cross sections elevations
cross_sections_elevations_df = HdfXsec.cross_sections_elevations(geom_hdf_path, ras_object=bald_eagle)
print("\nCross Sections Elevations:")
if not cross_sections_elevations_df.empty:
    display(cross_sections_elevations_df.head())
else:
    print("No cross section elevation data found in the geometry file.")


In [ ]:
# Example: Extract Refinement Regions
print("\nExample: Extracting Refinement Regions")

# Make sure to pass the bald_eagle object as the ras_object parameter
refinement_regions_df = HdfBndry.refinement_regions(geom_hdf_path, ras_object=bald_eagle)

if not refinement_regions_df.empty:
    print("Refinement Regions DataFrame:")
    display(refinement_regions_df.head())
    
    # Plot refinement regions
    fig, ax = plt.subplots(figsize=(12, 8))
    refinement_regions_df.plot(ax=ax, column='CellSize', legend=True, 
                               legend_kwds={'label': 'Cell Size', 'orientation': 'horizontal'},
                               cmap='viridis')
    ax.set_title('2D Mesh Area Refinement Regions')
    ax.set_xlabel('Easting')
    ax.set_ylabel('Northing')
    plt.tight_layout()
    plt.show()
else:
    print("No refinement regions found in the geometry file.")

# Example: Analyze Refinement Regions
if not refinement_regions_df.empty:
    print("\nRefinement Regions Analysis:")
    print(f"Total number of refinement regions: {len(refinement_regions_df)}")
    print("\nCell Size Statistics:")
    print(refinement_regions_df['CellSize'].describe())
    
    # Group by Shape Type
    shape_type_counts = refinement_regions_df['ShapeType'].value_counts()
    print("\nRefinement Region Shape Types:")
    print(shape_type_counts)
    
    # Plot Shape Type distribution
    plt.figure(figsize=(10, 6))
    shape_type_counts.plot(kind='bar')
    plt.title('Distribution of Refinement Region Shape Types')
    plt.xlabel('Shape Type')
    plt.ylabel('Count')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

In [ ]:
# Extract Compute Messages as String
print("Extracting Compute Messages")

import h5py
import numpy as np

def extract_string_from_hdf(results_hdf_filename: str, hdf_path: str) -> str:
    """
    Extract string from HDF object at a given path

    Parameters
    ----------
    results_hdf_filename : str
        Name of the HDF file
    hdf_path : str
        Path of the object in the HDF file

    Returns
    -------
    str
        Extracted string from the specified HDF object
    """
    with h5py.File(results_hdf_filename, 'r') as hdf_file:
        try:
            hdf_object = hdf_file[hdf_path]
            if isinstance(hdf_object, h5py.Group):
                return f"Group: {hdf_path}\nContents: {list(hdf_object.keys())}"
            elif isinstance(hdf_object, h5py.Dataset):
                data = hdf_object[()]
                if isinstance(data, bytes):
                    return data.decode('utf-8')
                elif isinstance(data, np.ndarray) and data.dtype.kind == 'S':
                    return [v.decode('utf-8') for v in data]
                else:
                    return str(data)
            else:
                return f"Unsupported object type: {type(hdf_object)}"
        except KeyError:
            return f"Path not found: {hdf_path}"

try:
    results_summary_string = extract_string_from_hdf(plan_hdf_path, '/Results/Summary/Compute Messages (text)')
    print("Compute Messages:")
    
    # Parse and print the compute messages in a more visually friendly way
    messages = results_summary_string[0].split('\r\n')
    
    for message in messages:
        if message.strip():  # Skip empty lines
            if ':' in message:
                key, value = message.split(':', 1)
                print(f"{key.strip():40} : {value.strip()}")
            else:
                print(f"\n{message.strip()}")
    
    # Print computation summary in a table format
    print("\nComputation Summary:")
    print("-" * 50)
    print(f"{'Computation Task':<30} {'Time':<20}")
    print("-" * 50)
    for line in messages:
        if 'Computation Task' in line:
            task, time = line.split('\t')
            print(f"{task:<30} {time:<20}")
    
    print("\nComputation Speed:")
    print("-" * 50)
    print(f"{'Task':<30} {'Simulation/Runtime':<20}")
    print("-" * 50)
    for line in messages:
        if 'Computation Speed' in line:
            task, speed = line.split('\t')
            print(f"{task:<30} {speed:<20}")

except Exception as e:
    print(f"Error extracting compute messages: {str(e)}")
    print("\nNote: If 'Results/Summary Output' is not in the file structure, it might indicate that the simulation didn't complete successfully or the results weren't saved properly.")

 



In [ ]:
# Advanced Compute Messages Example - TODO: Move this function into a class of the library 
import pandas as pd
import re
import matplotlib.pyplot as plt
import geopandas as gpd
import logging

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')

def parse_2d_compute_messages(compute_messages):
    """
    Parse 2D compute messages to extract data lines, clean the data, 
    and retrieve top 20 cells with the highest error.

    Parameters:
        compute_messages (list or str): The raw compute messages.

    Returns:
        tuple: A tuple containing the parsed compute messages string and the main DataFrame.
    """
    try:
        # Handle both list and string inputs
        if isinstance(compute_messages, list):
            compute_messages = '\n'.join(compute_messages)
        elif not isinstance(compute_messages, str):
            logging.error(f"Unexpected type for compute_messages: {type(compute_messages)}")
            return "", pd.DataFrame()

        # Split the message into lines
        lines = compute_messages.split('\n')
        logging.info("Successfully split compute messages into lines.")
        
        # Initialize lists to store parsed data
        data_lines = []
        header_lines = []
        footer_lines = []
        
        # Regular expression to match timestamp lines
        timestamp_pattern = re.compile(r'^\d{2}[A-Z]{3}\d{4}\s+\d{2}:\d{2}:\d{2}')
        logging.debug("Compiled timestamp regular expression.")
        
        data_started = False
        for line in lines:
            stripped_line = line.strip()
            if timestamp_pattern.match(stripped_line):
                data_started = True
                # Split the line and add to data_lines
                parts = stripped_line.split()
                if len(parts) >= 8:  # Ensure we have all expected columns
                    # Combine Date and Time into 'Date and Time'
                    date_time = f"{parts[0]} {parts[1]}"
                    location = parts[2]
                    cell_type = f"{parts[3]} {parts[4]}"
                    cell_number = parts[5]
                    wsel = parts[6]
                    error = parts[7]
                    iterations = parts[8] if len(parts) > 8 else None
                    data_lines.append([date_time, location, cell_type, cell_number, wsel, error, iterations])
                    logging.debug(f"Parsed data line: {data_lines[-1]}")
                else:
                    logging.warning(f"Line skipped due to insufficient parts: {stripped_line}")
            elif not data_started:
                header_lines.append(stripped_line)
            elif data_started and not stripped_line:
                data_started = False
            elif not data_started:
                footer_lines.append(stripped_line)
        
        # Create DataFrame from data lines
        df = pd.DataFrame(
            data_lines, 
            columns=['Date and Time', 'Location', 'Cell Type', 'Cell Number', 'WSEL', 'ERROR', 'ITERATIONS']
        )
        logging.info("Created DataFrame from parsed data lines.")
        
        # Clean and convert columns to appropriate types
        df['Cell Number'] = (
            pd.to_numeric(df['Cell Number'].replace('#', pd.NA), errors='coerce')
            .fillna(-1)
            .astype('Int64')
        )
        df['WSEL'] = pd.to_numeric(df['WSEL'], errors='coerce')
        df['ERROR'] = pd.to_numeric(df['ERROR'], errors='coerce')
        df['ITERATIONS'] = pd.to_numeric(df['ITERATIONS'], errors='coerce').astype('Int64')
        logging.info("Converted DataFrame columns to appropriate types.")
        
        # Get top 20 cells with highest error
        top_20_cells = (
            df.sort_values('ERROR', ascending=False)
            .drop_duplicates('Cell Number')
            .head(20)
        )
        
        # Construct the reordered message
        reordered_message = '\n'.join(header_lines + 
                                      ['\nTop 20 Cells with Highest Error:'] + 
                                      [' '.join(map(str, row)) for row in top_20_cells.values] + 
                                      ['\n'] + footer_lines)
        
        logging.info("Reordered compute messages.")
        
        return reordered_message, df
    except Exception as e:
        logging.error(f"Error parsing compute messages: {e}")
        return "", pd.DataFrame()

# Use the function to parse compute messages
parsed_messages, df = parse_2d_compute_messages(results_summary_string)

print(parsed_messages)
print(df)

# Get top 20 cells with highest error
if not df.empty and 'ERROR' in df.columns:
    top_20_cells = (
        df.sort_values('ERROR', ascending=False)
        .drop_duplicates('Cell Number')
        .head(20)
    )
else:
    logging.warning("Unable to get top 20 cells with highest error. DataFrame is empty or 'ERROR' column is missing.")
    top_20_cells = pd.DataFrame()

# Example: Get 2D Flow Area Perimeter Polygons (mesh_areas)
print("\nExample: Extracting 2D Flow Area Perimeter Polygons")
mesh_areas = HdfMesh.mesh_areas(geom_hdf_path, ras_object=bald_eagle)

print("\n2D Flow Area Groups and Perimeters:")
if not mesh_areas.empty:
    print("Available columns:", mesh_areas.columns.tolist())
    
    # Display the first few rows of the mesh_areas DataFrame
    print("\nFirst few rows of mesh_areas DataFrame:")
    display(mesh_areas.head())
else:
    print("No 2D Flow Area groups found in the HDF file.")

# Use the previously extracted cell_polygons_df
print("\nTop 20 Cell Polygons:")
if 'cell_polygons_df' in locals() and not cell_polygons_df.empty and not top_20_cells.empty:
    # Get the cell numbers from top_20_cells
    top_20_cell_numbers = top_20_cells['Cell Number'].tolist()
    
    # Filter cell_polygons_df to only include top 20 cells
    top_20_cell_polygons = cell_polygons_df[cell_polygons_df['cell_id'].isin(top_20_cell_numbers)]
    
    display(top_20_cell_polygons)

    # Plot top 20 cell polygons and mesh areas
    fig, ax = plt.subplots(figsize=(12, 8))
    
    # Plot mesh areas
    mesh_areas.plot(ax=ax, edgecolor='red', facecolor='none', alpha=0.5, label='Mesh Areas')
    
    # Plot top 20 cell polygons
    top_20_cell_polygons.plot(ax=ax, edgecolor='blue', facecolor='none', alpha=0.7, label='Top 20 Error Cells')
    
    # Set labels and title
    ax.set_xlabel('X Coordinate')
    ax.set_ylabel('Y Coordinate')
    ax.set_title('2D Flow Area Perimeters and Top 20 Cell Polygons')
    
    # Add legend
    ax.legend()
    
    # Add grid
    ax.grid(True)
    
    # Adjust layout and display
    plt.tight_layout()
    plt.show()
else:
    print("No Cell Polygons found or no top 20 cells with highest error available.")
    print("Unable to plot cell polygons.")

In [ ]:
# Exploratory Example for Debugging or New Features: List all paths, groups, and attributes under "/Results/Unsteady/Summary/Volume Accounting"
print("\nListing paths, groups, and attributes under '/Results/Unsteady/Summary/Volume Accounting'")

from ras_commander import HdfUtils

def list_hdf_structure(hdf_path: str, group_path: str) -> None:
    with h5py.File(hdf_path, 'r') as hdf:
        if group_path not in hdf:
            print(f"Group '{group_path}' not found in the HDF file.")
            return

        def print_group_structure(name: str, obj: h5py.Group) -> None:
            indent = '  ' * name.count('/')
            if isinstance(obj, h5py.Group):
                print(f"{indent}{name} (Group)")
                for attr_name, attr_value in obj.attrs.items():
                    print(f"{indent}  Attribute: {attr_name} = {attr_value}")
            elif isinstance(obj, h5py.Dataset):
                print(f"{indent}{name} (Dataset)")
                for attr_name, attr_value in obj.attrs.items():
                    print(f"{indent}  Attribute: {attr_name} = {attr_value}")

        hdf[group_path].visititems(print_group_structure)

try:
    list_hdf_structure(plan_hdf_path, "/Results/Unsteady/Summary/Volume Accounting")
except Exception as e:
    print(f"An error occurred while listing HDF structure: {str(e)}")

# Additional error handling and logging
logger = logging.getLogger(__name__)
logger.info("Finished listing HDF structure for Volume Accounting")


In [ ]:
# Example 12: Extract Plan Parameters and Volume Accounting
print("\nExample 12: Extracting Plan Parameters and Volume Accounting Data")

# Extract plan parameters
plan_parameters_df = HdfPlan.get_plan_param_attrs(plan_hdf_path)

# Extract volume accounting data
volume_accounting_df = HdfResultsPlan.get_results_volume_accounting_attrs(plan_hdf_path)

print("\nPlan Parameters DataFrame:")
display(plan_parameters_df)

print("\nVolume Accounting DataFrame:")
display(volume_accounting_df)

# RasPlanHdf Class Functions

In [ ]:
# Example: Get simulation start time
start_time = HdfPlan.get_simulation_start_time(plan_hdf_path)
print(f"Simulation start time: {start_time}")

In [ ]:
# Example: Get simulation end time
end_time = HdfPlan.get_simulation_end_time(plan_hdf_path)
print(f"Simulation end time: {end_time}")

In [ ]:
# Example: Extract 2D Flow Area Attributes 
print("\nExample 4: Extracting 2D Flow Area Attributes")
flow_area_attributes = HdfMesh.get_geom_2d_flow_area_attrs(geom_hdf_path, ras_object=bald_eagle)
if flow_area_attributes:
    # Convert the dictionary to a DataFrame for better display
    flow_area_attributes_df = pd.DataFrame([flow_area_attributes])
    display(flow_area_attributes_df)
else:
    print("No 2D Flow Area attributes found.")

In [ ]:
# Example: Get mesh max iterations
max_iter_df = HdfResultsMesh.mesh_max_iter(plan_hdf_path)
print("\nMesh Max Iterations:")
display(max_iter_df.head())

In [ ]:
# Example: Get mesh max water surface
max_ws_df = HdfResultsMesh.mesh_max_ws(plan_hdf_path, ras_object=bald_eagle)
print("\nMesh Max Water Surface:")
display(max_ws_df.head())

In [ ]:
# Example: Get mesh min water surface
min_ws_df = HdfResultsMesh.mesh_min_ws(plan_hdf_path, ras_object=bald_eagle)
print("\nMesh Min Water Surface:")
display(min_ws_df.head())

In [ ]:
# Example: Get mesh max face velocity
max_face_v_df = HdfResultsMesh.mesh_max_face_v(plan_hdf_path, ras_object=bald_eagle)
print("\nMesh Max Face Velocity:")
display(max_face_v_df.head())

In [ ]:
# Example: Get mesh min face velocity
min_face_v_df = HdfResultsMesh.mesh_min_face_v(plan_hdf_path, ras_object=bald_eagle)
print("\nMesh Min Face Velocity:")
display(min_face_v_df.head())

In [ ]:
# Example: Get mesh max water surface error
try:
    max_ws_err_df = HdfResultsMesh.mesh_max_ws_err(plan_hdf_path, ras_object=bald_eagle)
    print("\nMesh Max Water Surface Error:")
    display(max_ws_err_df.head())
except Exception as e:
    print(f"Error: {str(e)}")
    logger.error(f"Failed to get mesh max water surface error: {str(e)}")

In [ ]:
# Example: Get mesh summary output
try:
    summary_df = HdfResultsMesh.mesh_summary_output(plan_hdf_path, var="Maximum Water Surface", ras_object=bald_eagle)
    print("\nMesh Summary Output (Maximum Water Surface):")
    display(summary_df.head())
except Exception as e:
    print(f"Error: {str(e)}")
    logger.error(f"Failed to get mesh summary output: {str(e)}")
    # Additional error handling or logging can be added here

In [ ]:
# This is mesh summary output - not individual cell output.  This is best for small 1D or 2D storage areas

# Example: Get mesh summary output for Maximum Water Surface
summary_df_max_ws = HdfResultsMesh.mesh_summary_output(plan_hdf_path, var="Maximum Water Surface", ras_object=bald_eagle)
print("\nMesh Summary Output (Maximum Water Surface):")
display(summary_df_max_ws.head())

# Example: Get mesh summary output for Minimum Water Surface
summary_df_min_ws = HdfResultsMesh.mesh_summary_output(plan_hdf_path, var="Minimum Water Surface", ras_object=bald_eagle)
print("\nMesh Summary Output (Minimum Water Surface):")
display(summary_df_min_ws.head())

# Example: Get mesh summary output for Maximum Face Velocity
summary_df_max_fv = HdfResultsMesh.mesh_summary_output(plan_hdf_path, var="Maximum Face Velocity", ras_object=bald_eagle)
print("\nMesh Summary Output (Maximum Face Velocity):")
display(summary_df_max_fv.head())

# Example: Get mesh summary output for Minimum Face Velocity
summary_df_min_fv = HdfResultsMesh.mesh_summary_output(plan_hdf_path, var="Minimum Face Velocity", ras_object=bald_eagle)
print("\nMesh Summary Output (Minimum Face Velocity):")
display(summary_df_min_fv.head())

# Example: Get mesh summary output for Cell Maximum Water Surface Error
summary_df_max_ws_err = HdfResultsMesh.mesh_summary_output(plan_hdf_path, var="Cell Maximum Water Surface Error", ras_object=bald_eagle)
print("\nMesh Summary Output (Cell Maximum Water Surface Error):")
display(summary_df_max_ws_err.head())

# Example: Get mesh summary output for Cell Cumulative Iteration
summary_df_cum_iter = HdfResultsMesh.mesh_summary_output(plan_hdf_path, var="Cell Cumulative Iteration", ras_object=bald_eagle)
print("\nMesh Summary Output (Cell Cumulative Iteration):")
display(summary_df_cum_iter.head())

# Example: Get mesh summary output for Cell Last Iteration
summary_df_last_iter = HdfResultsMesh.mesh_summary_output(plan_hdf_path, var="Cell Last Iteration", ras_object=bald_eagle)
print("\nMesh Summary Output (Cell Last Iteration):")
display(summary_df_last_iter.head())


In [ ]:
# Example: Get mesh faces summary output
faces_summary_df = HdfMesh.mesh_cell_faces(plan_hdf_path, ras_object=bald_eagle)
print("\nMesh Faces Summary Output:")
display(faces_summary_df.head())


In [ ]:
# Get mesh cell polygons using the updated HdfMesh class
cell_polygons_gdf = HdfMesh.mesh_cell_polygons(plan_hdf_path, ras_object=bald_eagle)
print("\nMesh Cell Polygons:")
display(cell_polygons_gdf.head())

In [ ]:
# Get mesh cell faces using the updated HdfMesh class
cell_faces_gdf = HdfMesh.mesh_cell_faces(plan_hdf_path, ras_object=bald_eagle)
print("\nMesh Cell Faces:")
display(cell_faces_gdf.head())

In [ ]:
# Plot Mesh Cell Faces

import matplotlib.pyplot as plt
import logging

def plot_mesh_cell_faces(gdf, title="Mesh Cell Faces"):
    fig, ax = plt.subplots(figsize=(12, 8))
    
    # Plot the linestrings
    gdf.plot(ax=ax, color='blue', linewidth=0.5)
    
    # Customize the plot
    ax.set_title(title)
    ax.set_xlabel('Easting')
    ax.set_ylabel('Northing')
    ax.axis('equal')
    
    # Add grid lines
    ax.grid(True, linestyle='--', alpha=0.7)
    
    plt.tight_layout()
    plt.show()

# Plot the mesh cell faces
plot_mesh_cell_faces(cell_faces_gdf)

# Log the plotting action
logging.info(f"Plotted {len(cell_faces_gdf)} mesh cell faces.")

In [ ]:
# Get simulation start time
simulation_start_time = HdfPlan.get_simulation_start_time(plan_hdf_path, ras_object=bald_eagle)
print("\nSimulation Start Time:")
print(simulation_start_time)

# Get simulation end time
simulation_end_time = HdfPlan.get_simulation_end_time(plan_hdf_path, ras_object=bald_eagle)
print("\nSimulation End Time:")
print(simulation_end_time)

# Calculate simulation duration
simulation_duration = simulation_end_time - simulation_start_time
print("\nSimulation Duration:")
print(simulation_duration)

In [ ]:
# Get mesh timeseries output

# Get mesh areas from previous code cell
mesh_areas = HdfMesh.mesh_area_names(geom_hdf_path, ras_object=bald_eagle)

if mesh_areas:
    mesh_name = mesh_areas[0]  # Use the first 2D flow area name
    timeseries_da = HdfResultsMesh.mesh_timeseries_output(plan_hdf_path, mesh_name, "Water Surface", ras_object=bald_eagle)
    print(f"\nMesh Timeseries Output (Water Surface) for {mesh_name}:")
    print(timeseries_da)
else:
    print("No mesh areas found in the geometry file.")

In [ ]:
# Plot the time series
import matplotlib.pyplot as plt
import numpy as np

# Extract data from the DataArray
time = timeseries_da.time.values
water_surface = timeseries_da.values

# Calculate mean water surface elevation for each time step
mean_water_surface = np.mean(water_surface, axis=1)

# Create the plot
plt.figure(figsize=(12, 6))
plt.plot(time, mean_water_surface, label='Mean Water Surface Elevation')
plt.title(f'Mean Water Surface Elevation Over Time for {mesh_name}')
plt.xlabel('Time')
plt.ylabel('Water Surface Elevation (ft)')
plt.legend()
plt.grid(True)

# Rotate x-axis labels for better readability
plt.xticks(rotation=45, ha='right')

# Adjust layout and display the plot
plt.tight_layout()
plt.show()

# Log the plotting action
logging.info(f"Plotted mean water surface elevation time series for {mesh_name}")

In [61]:
# Time Series Output Variables for Cells
# 
# Variable Name: Description
# Water Surface: Water surface elevation
# Depth: Water depth
# Velocity: Magnitude of velocity
# Velocity X: X-component of velocity
# Velocity Y: Y-component of velocity
# Froude Number: Froude number
# Courant Number: Courant number
# Shear Stress: Shear stress on the bed
# Bed Elevation: Elevation of the bed
# Precipitation Rate: Rate of precipitation
# Infiltration Rate: Rate of infiltration
# Evaporation Rate: Rate of evaporation
# Percolation Rate: Rate of percolation
# Groundwater Elevation: Elevation of groundwater
# Groundwater Depth: Depth to groundwater
# Groundwater Flow: Groundwater flow rate
# Groundwater Velocity: Magnitude of groundwater velocity
# Groundwater Velocity X: X-component of groundwater velocity
# Groundwater Velocity Y: Y-component of groundwater velocity
# 
# These variables are available for time series output at the cell level in 2D flow areas.


In [ ]:
# Get mesh cells timeseries output
cells_timeseries_ds = HdfResultsMesh.mesh_cells_timeseries_output(plan_hdf_path, mesh_name, ras_object=bald_eagle)
print("\nMesh Cells Timeseries Output:")
print(cells_timeseries_ds)


In [ ]:
# Plot Cell Time Series Data (Random Cell ID)
import matplotlib.pyplot as plt
import numpy as np
import random

# Extract Water Surface data
water_surface = cells_timeseries_ds['BaldEagleCr']['Water Surface']

# Get the time values
time_values = water_surface.coords['time'].values

# Pick a random cell_id
random_cell_id = random.choice(water_surface.coords['cell_id'].values)

# Extract the water surface elevation time series for the random cell
wsel_timeseries = water_surface.sel(cell_id=random_cell_id)

# Find the peak value and its index
peak_value = wsel_timeseries.max().item()
peak_index = wsel_timeseries.argmax().item()

# Create the plot
plt.figure(figsize=(12, 6))
plt.plot(time_values, wsel_timeseries, label=f'Cell ID: {random_cell_id}')
plt.scatter(time_values[peak_index], peak_value, color='red', s=100, zorder=5)
plt.annotate(f'Peak: {peak_value:.2f} ft', 
             (time_values[peak_index], peak_value),
             xytext=(10, 10), textcoords='offset points',
             ha='left', va='bottom',
             bbox=dict(boxstyle='round,pad=0.5', fc='yellow', alpha=0.5),
             arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=0'))

plt.title(f'Water Surface Elevation Time Series for Random Cell (ID: {random_cell_id})')
plt.xlabel('Time')
plt.ylabel('Water Surface Elevation (ft)')
plt.legend()
plt.grid(True)
plt.tight_layout()

# Log the plotting action
logging.info(f"Plotted water surface elevation time series for random cell ID: {random_cell_id}")

# Display the plot
plt.show()

# Print some statistics
print(f"Statistics for Cell ID {random_cell_id}:")
print(f"Minimum WSEL: {wsel_timeseries.min().item():.2f} ft")
print(f"Maximum WSEL: {peak_value:.2f} ft")
print(f"Mean WSEL: {wsel_timeseries.mean().item():.2f} ft")
print(f"Time of peak: {time_values[peak_index]}")



In [ ]:
# Get mesh faces timeseries output
faces_timeseries_ds = HdfResultsMesh.mesh_faces_timeseries_output(plan_hdf_path, mesh_name, ras_object=bald_eagle)
print("\nMesh Faces Timeseries Output:")
print(faces_timeseries_ds)


In [ ]:
# Plot Random Cell Results and Label Peak

# Step 1: Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt

# Step 2: Select a random valid cell number
# Use .sizes instead of .dims to avoid FutureWarning
random_cell = np.random.randint(0, faces_timeseries_ds.sizes['cell'])

# Step 3: Extract time series data for the selected cell
variable = 'face_velocity'  # We could also use 'face_flow'
cell_data = faces_timeseries_ds[variable].sel(cell=random_cell)

# Step 4: Find peak value and its corresponding time
peak_value = cell_data.max().item()
peak_time = cell_data.idxmax().values

# Step 5: Create the plot
plt.figure(figsize=(12, 6))
plt.plot(faces_timeseries_ds.time, cell_data)
plt.title(f'{variable.capitalize()} Time Series for Cell {random_cell}')
plt.xlabel('Time')
plt.ylabel(f'{variable.capitalize()} ({faces_timeseries_ds.attrs["units"]})')
plt.grid(True)

# Step 6: Annotate the peak point
plt.annotate(f'Peak: ({peak_time}, {peak_value:.2f})', 
             (peak_time, peak_value),
             xytext=(10, 10), textcoords='offset points',
             arrowprops=dict(arrowstyle="->"))

# Step 7: Check for negative values and label the minimum if present
min_value = cell_data.min().item()
if min_value < 0:
    min_time = cell_data.idxmin().values
    plt.annotate(f'Min: ({min_time}, {min_value:.2f})', 
                 (min_time, min_value),
                 xytext=(10, -10), textcoords='offset points',
                 arrowprops=dict(arrowstyle="->"))

# Step 8: Display the plot
plt.tight_layout()
plt.show()

# Step 9: Print summary information
print(f"Random Cell: {random_cell}")
print(f"Peak Value: {peak_value:.2f} {faces_timeseries_ds.attrs['units']} at {peak_time}")
if min_value < 0:
    print(f"Minimum Value: {min_value:.2f} {faces_timeseries_ds.attrs['units']} at {min_time}")

In [ ]:
# Get reference lines
ref_lines_gdf = HdfBndry.reference_lines(plan_hdf_path, ras_object=bald_eagle)
print("\nReference Lines:")
display(ref_lines_gdf.head())

In [ ]:
# Get reference points
ref_points_gdf = HdfBndry.reference_points(plan_hdf_path, ras_object=bald_eagle)
print("\nReference Points:")
display(ref_points_gdf.head())

In [ ]:
# Get reference timeseries output
ref_timeseries_ds = HdfResultsPlan.reference_timeseries_output(plan_hdf_path, ras_object=bald_eagle)
print("\nReference Timeseries Output:")
print(ref_timeseries_ds)

In [ ]:
# Get reference lines timeseries output
ref_lines_timeseries_ds = HdfResultsPlan.reference_lines_timeseries_output(plan_hdf_path, ras_object=bald_eagle)
print("\nReference Lines Timeseries Output:")
print(ref_lines_timeseries_ds)

In [ ]:
# Get reference points timeseries output
ref_points_timeseries_ds = HdfResultsPlan.reference_points_timeseries_output(plan_hdf_path, ras_object=bald_eagle)
print("\nReference Points Timeseries Output:")
print(ref_points_timeseries_ds)

In [ ]:
# Get reference summary output
ref_summary_df = HdfResultsPlan.reference_summary_output(plan_hdf_path, ras_object=bald_eagle)
print("\nReference Summary Output:")
display(ref_summary_df.head())

In [ ]:
# Get meteorology precipitation attributes
meteo_precip_attrs = HdfPlan.get_meteorology_precip_attrs(plan_hdf_path, ras_object=bald_eagle)
print("\nMeteorology Precipitation Attributes:")
for key, value in meteo_precip_attrs.items():
    print(f"{key}: {value}")

In [ ]:
# Get results unsteady attributes
results_unsteady_attrs = HdfResultsPlan.get_results_unsteady_attrs(plan_hdf_path, ras_object=bald_eagle)
print("\nResults Unsteady Attributes:")
for key, value in results_unsteady_attrs.items():
    print(f"{key}: {value}")

In [ ]:
# Get results unsteady summary attributes
results_unsteady_summary_attrs = HdfResultsPlan.get_results_unsteady_summary_attrs(plan_hdf_path, ras_object=bald_eagle)
print("\nResults Unsteady Summary Attributes:")
for key, value in results_unsteady_summary_attrs.items():
    print(f"{key}: {value}")

# Get results volume accounting attributes
volume_accounting_attrs = HdfResultsPlan.get_results_volume_accounting_attrs(plan_hdf_path, ras_object=bald_eagle)
print("\nVolume Accounting Attributes:")
for key, value in volume_accounting_attrs.items():
    print(f"{key}: {value}")

In [ ]:
# Get steady profile cross-section output
steady_xs_df = HdfResultsXsec.steady_profile_xs_output(plan_hdf_path, "Water Surface", ras_object=bald_eagle)
print("\nSteady Profile Cross-Section Output (Water Surface):")
display(steady_xs_df.head())

In [ ]:
# Get cross-sections water surface elevation
xs_wsel_df = HdfResultsXsec.cross_sections_wsel(plan_hdf_path, ras_object=bald_eagle)
print("\nCross-Sections Water Surface Elevation:")
display(xs_wsel_df.head())

In [ ]:
# Get cross-sections flow
xs_flow_df = HdfResultsXsec.cross_sections_flow(plan_hdf_path, ras_object=bald_eagle)
print("\nCross-Sections Flow:")
display(xs_flow_df.head())


In [ ]:
# Get cross-sections energy grade
xs_energy_grade_df = HdfResultsXsec.cross_sections_energy_grade(plan_hdf_path, ras_object=bald_eagle)
print("\nCross-Sections Energy Grade:")
display(xs_energy_grade_df.head())

In [ ]:
# Get cross-sections additional encroachment station left
xs_enc_left_df = HdfResultsXsec.cross_sections_additional_enc_station_left(plan_hdf_path, ras_object=bald_eagle)
print("\nCross-Sections Additional Encroachment Station Left:")
display(xs_enc_left_df.head())

In [ ]:
# Get cross-sections additional encroachment station right
xs_enc_right_df = HdfResultsXsec.cross_sections_additional_enc_station_right(plan_hdf_path, ras_object=bald_eagle)
print("\nCross-Sections Additional Encroachment Station Right:")
display(xs_enc_right_df.head())

In [ ]:
# Get cross-sections additional area total
xs_area_total_df = HdfResultsXsec.cross_sections_additional_area_total(plan_hdf_path, ras_object=bald_eagle)
print("\nCross-Sections Additional Area Total:")
display(xs_area_total_df.head())


In [ ]:
# Get cross-sections additional velocity total
xs_velocity_total_df = HdfResultsXsec.cross_sections_additional_velocity_total(plan_hdf_path, ras_object=bald_eagle)
print("\nCross-Sections Additional Velocity Total:")
display(xs_velocity_total_df.head())

In [ ]:
# HdfUtils Examples

# Example: Get attributes for a specific path
attrs = HdfUtils.get_attrs(plan_hdf_path, attr_path="/Results/Unsteady")
print("\nAttributes for /Results/Unsteady:")
for key, value in attrs.items():
    print(f"{key}: {value}")

In [ ]:
# Example: Get root attributes
root_attrs = HdfUtils.get_root_attrs(plan_hdf_path)
print("\nRoot Attributes:")
for key, value in root_attrs.items():
    print(f"{key}: {value}")